In [12]:
!pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/graphcore/graphcore-cloud-tools (to revision v0.3) to /tmp/pip-install-l_e9ipix/graphcore-cloud-tools_aeed2bb0efea48cfa68b46d2b3f5f7f3
  Running command git clone --filter=blob:none --quiet https://github.com/graphcore/graphcore-cloud-tools /tmp/pip-install-l_e9ipix/graphcore-cloud-tools_aeed2bb0efea48cfa68b46d2b3f5f7f3
  Running command git checkout -q 0af1fc805cfb8f93fe2e2407e32bfeff2dce1f55
  Resolved https://github.com/graphcore/graphcore-cloud-tools to commit 0af1fc805cfb8f93fe2e2407e32bfeff2dce1f55
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.10.2-py3-none-any.whl (2.8 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully un

In [13]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [14]:
!pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.40.2-py3-none-any.whl (9.0 MB)
  Using cached tokenizers-0.19.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.10.2
    Uninstalling transformers-4.10.2:
      Successfully uninstalled transformers-4.10.2

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [15]:
#Loading the Hugging Face Dataset
from datasets import load_dataset

dataset1 = load_dataset("shrms/nomal_bar", split="train")

In [16]:
# 100 pieces of sample data for quick testing
cnt = 0
dataset = []
for i in dataset1:
    cnt+=1
    dataset.append(i)
    if cnt == 100:
        break
print(dataset[0]["image_file"]) #The image is stored in my local area
print(dataset[0]["text"])

In [18]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

MAX_PATCHES = 1024

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image_path = item["image_file"]
        image_path = image_path + ".jpg"
        image = Image.open(image_path).convert("RGB")

        if image is not None:
            encoding = self.processor(images=image,
                                      text="Generate underlying data table of the figure below:",
                                      return_tensors="pt",
                                      add_special_tokens=True,
                                      max_patches=MAX_PATCHES,
                                      padding=True,
                                      truncation=True)
            encoding = {k: v.squeeze() for k, v in encoding.items()}
            encoding["text"] = item["text"]
            return encoding
        
        else:
            print("Failed :",image)
            return None

In [19]:
# Get a great model

from transformers import AutoProcessor, Pix2StructForConditionalGeneration
import poptorch

# The model size changes during this process.. (layer replicated)
processor = AutoProcessor.from_pretrained("google/deplot")
model_me = Pix2StructForConditionalGeneration.from_pretrained("google/deplot")


# Optimized for IPU
opts = poptorch.Options()
opts.deviceIterations(1)
optimizer = poptorch.optim.SGD(model_me.parameters(), lr=1e-5, momentum=0.9, weight_decay=1e-5)
model = poptorch.trainingModel(model=model_me, options=opts, optimizer=optimizer)

print("Call successful")

호출 완료


In [20]:
def collator(batch):
    new_batch = {"flattened_patches": [], "attention_mask": []}
    texts = [item["text"] for item in batch]
    
    text_inputs = processor.tokenizer(text=texts,
                                      return_tensors="pt",
                                      add_special_tokens=True,
                                      max_length=512, 
                                      padding=True,
                                      truncation=True)
    new_batch["labels"] = text_inputs.input_ids

    for item in batch:
        new_batch["flattened_patches"].append(item["flattened_patches"])
        new_batch["attention_mask"].append(item["attention_mask"])
    
    new_batch["flattened_patches"] = torch.stack(new_batch["flattened_patches"])
    new_batch["attention_mask"] = torch.stack(new_batch["attention_mask"])
    return new_batch

In [21]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = poptorch.DataLoader(dataset=train_dataset, options=opts, shuffle=True, batch_size=1, collate_fn=collator)
#Even if you adjust the batch size to 1, the kernel will die
#I think the replication of the model structure increases the amount of memory required, and so the kernel dies

In [ ]:
# Create Model Storage Location
modeltarget = "save_model_1"

import os
os.makedirs("./model/"+modeltarget, exist_ok=True)

limit = 5000

In [ ]:
import torch
EPOCHS = 10

# model_me.train()
for epoch in range(EPOCHS):
    print("Epoch:", epoch)
    
    model.train()
    processor.image_processor.is_vqa = False
    for idx, batch in enumerate(train_dataloader):
        flattened_patches = batch.pop("flattened_patches").to(torch.float32)
        attention_mask = batch.pop("attention_mask").to(torch.float32)
        labels = batch.pop("labels").to(torch.int32)

        outputs = model(flattened_patches=flattened_patches,
                        attention_mask=attention_mask,
                        labels=labels)
        
        loss = outputs.loss
        
        print("Loss:", loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 20 == 0:
            model_me.eval()
        
            predictions = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask)        
            print("Predictions:", processor.batch_decode(predictions, skip_special_tokens=True))
        
            model.train()
        
model.save_pretrained("./model/"+modeltarget)
torch.save(model.state_dict(), "./model/"+modeltarget+"/model.pt")